In [1]:
using PyCall
using Statistics
using LinearAlgebra
using Random
using Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
using JLD2
#import tt_option_greeks as ttog
;

In [2]:
# PyCallを用いてPythonのライブラリをインポート
np = pyimport("numpy")
@pyimport tf_quant_finance as tff
@pyimport tensorflow as tf

In [3]:
# オプション満期
T = 1.0
# 無リスク金利
r = 0.01
# strike
K = 100.0
# 原資産数
d = 5
# 原資産価格初期値

corrMat = Matrix{Float64}(I, d, d)
for i in 1:d
    for j in 1:d
        if i != j
            corrMat[i, j] = 1/2
        end
    end
end
nPath = 10^6
S0 = fill(100, d)
;

In [4]:
function chebyshev_lobatto_nodes(n::Int, a::Real, b::Real)
    # 標準区間 [-1,1] を -1 → +1 の順で生成
    x_standard = [cos(π * (n - 1 - i) / (n - 1)) for i in 0:n-1]

    # [a,b] への線形変換: x \in [-1,1] を x_scaled \in [a,b] に写す
    #   x_scaled = (b-a)/2 * (x_standard + 1) + a
    x_scaled   = 0.5 * (b - a) .* (x_standard .+ 1) .+ a

    return x_scaled, x_standard
end

σs, _ =  chebyshev_lobatto_nodes(100, 0.15, 0.25)
len_σs = length(σs)
stock0, _ =  chebyshev_lobatto_nodes(100, 90, 120)
len_stock0 = length(stock0)
len = len_σs * len_stock0

10000

In [5]:



corrMat = [
    1.0       0.303659  0.505532  0.719655  0.728832;
 0.303659  1.0       0.401077  0.515272  0.178203;
 0.505532  0.401077  1.0       0.132542  0.722767;
 0.719655  0.515272  0.132542  1.0       0.394723;
 0.728832  0.178203  0.722767  0.394723  1.0]

#new_order = [1, 4, 5, 3, 2]
#reordered_corrMat = corrMat[new_order, new_order]

5×5 Matrix{Float64}:
 1.0       0.303659  0.505532  0.719655  0.728832
 0.303659  1.0       0.401077  0.515272  0.178203
 0.505532  0.401077  1.0       0.132542  0.722767
 0.719655  0.515272  0.132542  1.0       0.394723
 0.728832  0.178203  0.722767  0.394723  1.0

In [6]:
function calculate_option_price(T::Float64, r::Float64, K::Float64, d::Int64, S0::Vector{Int64}, stock0::Vector{Float64}, localindex_v::Vector{Int64}, nPath::Int64, σs::Vector{Float64}, corrMat)::Tuple{Float64, Float64}
    vols = [σs[i] for i in localindex_v[1:2:2*d]]
    S0s = [stock0[i] for i in localindex_v[2:2:2*d]]
    process = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        volatilities=vols,
        corr_matrix=corrMat)
    
    paths = process.sample_paths(
        times=[T],
        initial_state=S0s,
        random_type=tff.math.random.RandomType.PSEUDO,
        num_samples=nPath).numpy()
    
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    #payoffs = vec(payoffs)  # 配列をベクトルに変換
    mean_pv = exp(-r * T) * mean(payoffs)
    std_pv = exp(-r * T) * std(payoffs)
    return (mean_pv, std_pv)
end

calculate_option_price (generic function with 1 method)

In [8]:
# オプション価格の計算
localindex_v = [100 for _ in 1:2d]
mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
println("Present Value: ", mean_pv)
println("Standard Deviation: ", std_pv)
println("Abs.Err.: ", std_pv/sqrt(nPath))
#pv = calculate_option_price(T, r, K, d, S0, localindex, nPath, σs, corrMat)
#println(pv)
;

Present Value: 7.0657811973786275
Standard Deviation: 12.397851604560513
Abs.Err.: 0.012397851604560512


In [13]:
using Statistics, Random

n_trials = 100
price = Float64[]
err = Float64[]

for _ in 1:n_trials
    # 100個のランダムなlocalindex_vを作成（2d要素、各要素は例えば 1〜200 の範囲）
    localindex_v = [rand(1:100) for _ in 1:2d]
    
    mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
    
    #if std_pv != 0.0
    #    push!(ratios, mean_pv / std_pv)
    #end
    push!(price, mean_pv)
    push!(err, std_pv /sqrt(nPath) )
end

# 平均値を出力
avg_ratio = mean(err ./ maximum(price))
println("平均(mean_pv / std_pv): ", avg_ratio)


平均(mean_pv / std_pv): 0.0006936873303209204


In [16]:
tol = 0.0006936873303209204
tol = 1.0 * 1e-3

0.001

In [28]:
function generate_random_combinations(len, d, num_samples)
    Random.seed!(1234)

    possible_values = collect(1:len)
    random_combinations = Vector{Int}[]
    
    for _ in 1:num_samples
        combination = rand(possible_values, d)
        push!(random_combinations, combination)
    end
    
    return random_combinations
end


# Option pricing taking random combinations of parameters
function rand_inner(T, r, K, d, S0, stock0, nPath, σs, corrMat, len, num_samples, random_combinations)
    #random_combinations = generate_random_combinations(len, d, num_samples)
    #random_combinations = [[i, 2, 2, 2, 2] for i in 1:len]
    results = Vector{ComplexF64}()
    for i in 1:num_samples
        ind = random_combinations[i]
        #indices = ttog.indices_1d_to_2d(localindex, 100)
        localindex_v = ttog.indices_1d_to_2d(ind, 100)
        #@show localindex_v
        #localindex_v = ind
        #inner_ = inner_p(d, ind, M_phi, M_v)
        #result = (( η^d ) * exp(-r * (T - t)) * inner_) / ((2 * π) ^ d)
        result, err = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
        push!(results, result)
    end
    return results, random_combinations
end

rand_inner (generic function with 1 method)

In [29]:
num_samples = 100  # Number of parameter combinations
len


10000

In [30]:
random_combinations = generate_random_combinations(len, d, num_samples);

In [31]:
results, combinations = rand_inner(T, r, K, d, S0, stock0, nPath, σs, corrMat, len, num_samples, random_combinations)
#results, combinations = rand_inner(T, r, K, d, S0, stock0, localindex_v, nPath, σs, reordered_corrMat, len, num_samples, random_combinations)

mc = real(results)
#JLD2.@save "vol&stock0_mc_d5_50000000_const_extended.jld2" mc
mc

UndefVarError: UndefVarError: `ttog` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
mc

UndefVarError: UndefVarError: `mc` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
#JLD2.@save "vol&stock0_mc_d5_50000000_rand_reordered.jld2" mc
#JLD2.@save "vol&stock0_mc_d5_50000000_rand_cheby.jld2" mc
JLD2.@save "vol&stock0_mc_d5_100000000_const_noise_cheby_gr100.jld2" mc

LoadError: LoadError: UndefVarError: `JLD2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at /Users/sakurairihito/Documents/Research/post-doc/Tensor_FixedSeedMC/notebook/multi-asset/Europian_min_op/MC/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:3